In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# # === Clone HyperThelia if needed===

from pathlib import Path
from lib.setup_functions import clone_hyperthelia_repo, setup_project_io

CLONE_PARENT_DIR = Path("/content")
CLONE_DIR, BASE_PROJECT_DIR = clone_hyperthelia_repo(CLONE_PARENT_DIR)


In [ ]:
from pathlib import Path

# === BASE PROJECT SETUP ===
BASE_PROJECT_DIR = Path("/content/HyperThelia/Hyperthelia_project")
RAW_DIR = Path("/content/HyperThelia/Hyperthelia_project/data_demo")



In [ ]:
# # === SESSION SETUP IMPORTS AND FUNCTIONS ===
RAW_DIR, OUTPUTS_DIR = setup_project_io(BASE_PROJECT_DIR)
import segmentation
importlib.reload(segmentation)

# Install latest Cellpose from GitHub (includes SAM support)
!pip install git+https://github.com/mouseland/cellpose.git

In [ ]:
# === LOOK FOR EXPERIMENTS IN RAW_DIR ===
from segmentation import print_experiment_summary
print_experiment_summary(RAW_DIR)


In [ ]:
from segmentation import run_segmentation_pipeline

# === SEGMENTATION PARAMETERS ===
Z_AXIS = 0
CHANNEL_AXIS = 1
BATCH_SIZE = 32
DO_3D = False
STITCH_THRESHOLD = 0.5

# === RUN PIPELINE ===
run_segmentation_pipeline(
    RAW_DIR,
    OUTPUTS_DIR,
    model,
    z_axis=Z_AXIS,
    channel_axis=CHANNEL_AXIS,
    batch_size=BATCH_SIZE,
    do_3D=DO_3D,
    stitch_threshold=STITCH_THRESHOLD
)


In [ ]:
from measurevisualise import interactive_segmentation_viewer
interactive_segmentation_viewer(BASE_PROJECT_DIR / "outputs")

import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from tifffile import imread
from pathlib import Path

# === Call Viewer with your segmentation outputs directory
interactive_segmentation_viewer(BASE_PROJECT_DIR / "outputs")
